## Introduction to Quimb backend in QiboTN

#### Some imports

In [1]:
import time
import numpy as np
# from scipy import stats

# import qibo
from qibo import Circuit, gates, hamiltonians
from qibo.backends import construct_backend

#### Some hyper parameters

In [2]:
import cotengra as ctg
ctg_opt = ctg.ReusableHyperOptimizer(
    max_time=10,
    minimize='combo',
    slicing_opts=None,
    parallel=True,
    progbar=True
)


In [3]:
# construct qibotn backend
quimb_backend = construct_backend(backend="qibotn", platform="quimb")

# set number of qubits
nqubits = 4

# set numpy random seed
np.random.seed(42)

quimb_backend.setup_backend_specifics(quimb_backend="jax")

#### Constructing a parametric quantum circuit

In [4]:
def build_circuit(nqubits, nlayers):
    """Construct a parametric quantum circuit."""
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ

In [5]:
circuit = build_circuit(nqubits=nqubits, nlayers=3)
circuit.draw()

0: ─RY─RZ─o─────X─RY─RZ─o─────X─RY─RZ─o─────X─M─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─RY─RZ─X─o───|─M─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─RY─RZ───X─o─|─M─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─RY─RZ─────X─o─M─


In [6]:
# Setting random parameters
circuit.set_parameters(
    parameters=np.random.uniform(-np.pi, np.pi, len(circuit.get_parameters())),
)

#### Setting up the tensor network simulator

Depending on the simulator, various parameters can be set. One can customize the tensor network execution via the `backend.configure_tn_simulation` function, whose face depends on the specific backend provider.

In [7]:
# Customization of the tensor network simulation in the case of quimb backend
# Here we use only some of the possible arguments
quimb_backend.configure_tn_simulation(
    #ansatz="MPS",
    max_bond_dimension=10
)

#### Executing through the backend

The `backend.execute_circuit` method can be used then. We can simulate results in three ways:
1. reconstruction of the final state only if `return_array` is set to `True`;
2. computation of the relevant probabilities of the final state.
3. reconstruction of the relevant state's frequencies (only if `nshots` is not `None`).

In [8]:
# # Simple execution (defaults)
outcome = quimb_backend.execute_circuit(circuit=circuit, nshots=100, return_array=True)

# # Print outcome
vars(outcome)

/home/andrea/python_envs/3.11/lib/python3.11/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: creg
  warnings.warn(
/home/andrea/python_envs/3.11/lib/python3.11/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: measure
  warnings.warn(


{'nqubits': 4,
 'backend': qibotn (quimb),
 'measures': Counter({'1101': 14,
          '1000': 12,
          '0010': 11,
          '0011': 11,
          '0110': 9,
          '0000': 8,
          '1010': 7,
          '1110': 6,
          '0100': 5,
          '1111': 5,
          '1011': 5,
          '0101': 4,
          '0111': 1,
          '0001': 1,
          '1100': 1}),
 'measured_probabilities': {'1101': np.float64(0.12331159869893284),
  '1000': np.float64(0.11330883548333684),
  '0010': np.float64(0.0946686048198943),
  '0011': np.float64(0.07571277233522157),
  '0110': np.float64(0.051460648073692314),
  '0000': np.float64(0.08390937969317334),
  '1010': np.float64(0.03872758515126775),
  '1110': np.float64(0.07174919872960006),
  '0100': np.float64(0.07142939529687146),
  '1111': np.float64(0.10184806171791994),
  '1011': np.float64(0.053499396925872716),
  '0101': np.float64(0.05622305772698606),
  '0111': np.float64(0.040291850747292815),
  '0001': np.float64(0.00467701119520

---

One can access to the specific contents of the simulation outcome.

In [9]:
print(f"Probabilities:\n {outcome.probabilities()}\n")
print(f"State:\n {outcome.state()}\n")

Probabilities:
 {'1101': np.float64(0.12331159869893284), '1000': np.float64(0.11330883548333684), '0010': np.float64(0.0946686048198943), '0011': np.float64(0.07571277233522157), '0110': np.float64(0.051460648073692314), '0000': np.float64(0.08390937969317334), '1010': np.float64(0.03872758515126775), '1110': np.float64(0.07174919872960006), '0100': np.float64(0.07142939529687146), '1111': np.float64(0.10184806171791994), '1011': np.float64(0.053499396925872716), '0101': np.float64(0.05622305772698606), '0111': np.float64(0.040291850747292815), '0001': np.float64(0.004677011195208322), '1100': np.float64(0.013605984872668443)}

State:
 [[ 0.08809626-0.27595j   ]
 [-0.05174781+0.04471214j]
 [ 0.00470146+0.30764672j]
 [-0.27208942+0.04098931j]
 [ 0.18807825+0.1898841j ]
 [ 0.22377063+0.07842041j]
 [-0.18900302+0.12545316j]
 [ 0.17105258-0.10503745j]
 [ 0.24859732-0.22695422j]
 [-0.04117391-0.0623003j ]
 [ 0.17371394-0.09247189j]
 [-0.22748126+0.04185291j]
 [ 0.09444097+0.06846087j]
 [-0

### Compute expectation values

Another important feature of this backend is the `expectation` function. In fact, we can compute expectation values of given observables thorugh a Qibo-friendly interface.

---

Let's start by importing some symbols, thanks to which we can build our observable.

In [10]:
import numpy as np
import jax
from qibo.backends import construct_backend
from qibo import Circuit, gates

In [11]:
# construct qibotn backend
quimb_backend = construct_backend(backend="qibotn", platform="quimb")

quimb_backend.setup_backend_specifics(
    quimb_backend    ="jax", 
    contractions_optimizer='auto-hq'
    )

quimb_backend.configure_tn_simulation(
    max_bond_dimension=10
)

In [18]:
from qibo.symbols import X, Z, Y
from qibo.hamiltonians import XXZ

# define Hamiltonian
hamiltonian = XXZ(4, dense=False, backend=quimb_backend)

In [19]:
# define circuit
def build_circuit(nqubits, nlayers):
    circ = Circuit(nqubits)
    for layer in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
            circ.add(gates.RX(q=q, theta=0.))
        for q in range(nqubits - 1):
            circ.add(gates.CNOT(q, q + 1))
            circ.add(gates.SWAP(q, q + 1))
    circ.add(gates.M(*range(nqubits)))
    return circ

def build_circuit_problematic(nqubits, nlayers):
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ


nqubits = 4
circuit = build_circuit(nqubits=nqubits, nlayers=3)


In [20]:
start = time.time()
expval = hamiltonian.expectation(circuit)

elapsed = time.time() - start
print(f"Expectation value: {expval}")
print(f"Elapsed time: {elapsed:.4f} seconds")

Expectation value: 2.0
Elapsed time: 0.0268 seconds


Try with Qibo (which is by default using the Qibojit backend)


In [21]:
sym_hamiltonian = XXZ(4, dense=False, backend=None)

#  Let's show it
sym_hamiltonian.form

# Compute expectation value
start = time.time()
result = sym_hamiltonian.expectation(circuit().state())
elapsed = time.time() - start
print(f"Expectation value: {result}")
print(f"Elapsed time: {elapsed:.4f} seconds")

[Qibo 0.2.21|INFO|2025-10-27 16:24:00]: Using numpy backend on /CPU:0


Expectation value: 2.0
Elapsed time: 0.0360 seconds


They match! 🥳

We can also compute gradient of expectation function

In [23]:
def f(circuit, hamiltonian, params):
    circuit.set_parameters(params)
    return hamiltonian.expectation(
        circuit=circuit,
    )

parameters = np.random.uniform(-np.pi, np.pi, size=len(circuit.get_parameters()))
print(jax.grad(f, argnums=2)(circuit, hamiltonian, parameters))


/home/andrea/python_envs/3.11/lib/python3.11/site-packages/quimb/tensor/circuit.py:4927: UserWarning: Unsupported options for computing local_expectation with an MPS circuit supplied, ignoring: R, None, None, jax, None
  warnings.warn(
/home/andrea/python_envs/3.11/lib/python3.11/site-packages/quimb/tensor/circuit.py:4927: UserWarning: Unsupported options for computing local_expectation with an MPS circuit supplied, ignoring: R, None, None, jax, None
  warnings.warn(


[-0.24630009  0.8370421  -0.11103702 -0.12855841  0.41325414 -0.0628037
  0.51638705  0.794163   -0.27972788 -1.0718998   0.02731732  1.0153619
 -0.34494495  1.5744264   0.26920277 -0.36333832  0.12331417  0.5196531
  1.1294655   0.29257926 -0.18237355  0.8914014  -0.9471657   0.3492473
 -0.3477673   0.24325958  0.04818404 -0.87983793  0.47196424  0.36605012
  1.005       0.65054715 -0.94860053  0.14459445  0.36571163 -0.2550101 ]
